In [11]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Make Detections

In [45]:
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
  cap = cv2.VideoCapture(0)
  while cap.isOpened():
    ret, frame = cap.read()

    # Recolor image
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    
    # Make detection
    results = pose.process(image)
    # print(results)

    # Recolor image to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # Extract landmarks
    try:
      landmarks = results.pose_landmarks.landmark
      # print(landmarks)
    except:
      pass

    # Render detections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                              mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2),)
    
    cv2.imshow('Mediapipe feed', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
      break

  cap.release()
  cv2.destroyAllWindows()

In [46]:
len(landmarks)

33

In [47]:
for lndmark in mp_pose.PoseLandmark:
  print(lndmark)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [48]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.59611726
y: 0.8414963
z: -0.16593502
visibility: 0.9993169

In [49]:
landmarks[0]

x: 0.40018368
y: 0.49282682
z: -1.2067094
visibility: 0.9998638

# Calculate Angles

In [50]:
def calculate_angle(a,b,c):
  a = np.array(a)
  b = np.array(b)
  c = np.array(c)

  radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
  angle = np.abs(radians*180.0/np.pi)

  if angle>180.0:
    angle = 360 - angle

  return angle

In [51]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [52]:
shoulder, elbow, wrist

([0.5961172580718994, 0.8414962887763977],
 [0.9756803512573242, 0.9008592367172241],
 [0.7575255036354065, 0.6451485753059387])

In [53]:
calculate_angle(shoulder, elbow, wrist)

40.64251469443965